# My Evaluation Approach


![](assets/my_approach.png)


## Setup


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import time
import os
from typing import List, Dict, TypedDict
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import nest_asyncio
from dotenv import load_dotenv

import openai
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import VectorStore
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

In [17]:
loaded = load_dotenv(override=True)

data_dir = "my_benchmark/"
os.environ['CHUNKING_BENCHMARK'] = data_dir

# 1. Load and Save Documents


Each document is loaded as one Langchain document possibly to small to fit into a LLM. Therefore, we need to split these documents into smaller pieces of text for further processing.

In [23]:
from utils.loader import save_documents

documents: List[Document] = []
for file in os.listdir(data_dir+"documents"):
    file_path = os.path.join(data_dir+"documents", file)
    loader = TextLoader(file_path)
    documents.extend(loader.load())

save_documents(documents, data_dir)

In [4]:
from utils.loader import load_documents
documents = load_documents(data_dir)

# 2. Apply chunking


In [ ]:
%run -i chunking_strategies.ipynb

In [4]:
from utils.loader import load_chunks
split_chunks: Dict[str, Document] = load_chunks(data_dir)

# 3. Ingest into vector store

Using FAISS


In [32]:
from langchain_huggingface import HuggingFaceEmbeddings

vector_stores: Dict[str, VectorStore] = {}

# embeddings = HuggingFaceEmbeddings(
#     model_name="Snowflake/snowflake-arctic-embed-l",
#     model_kwargs={"device": 0, 'trust_remote_code': True},  # Comment out to use CPU
# )
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store_dir = f"{data_dir}vector_stores/{embeddings.model.replace('/', '-')}"
Path(vector_store_dir).mkdir(parents=True, exist_ok=True)
for experiment_name, chunks in split_chunks.items():
    if os.path.exists(f"{vector_store_dir}/{experiment_name}"):
        print("Loading", experiment_name)
        vector_stores[experiment_name] = FAISS.load_local(f"{vector_store_dir}/{experiment_name}", embeddings, allow_dangerous_deserialization=True)
    else:
        print("Indexing", experiment_name)
        vector_stores[experiment_name] = FAISS.from_documents(chunks, embeddings)
        vector_stores[experiment_name].save_local(f"{vector_store_dir}/{experiment_name}")

Indexing markdown-header-recursive-512-0
Indexing fixed-size-2048-0
Indexing markdown-header-recursive-1024-0
Indexing markdown-header-recursive-2048-200
Indexing recursive-1024-200
Indexing fixed-size-512-0
Indexing fixed-size-1024-0
Indexing markdown-header-recursive-1024-200
Indexing markdown-header-recursive-2048-0
Indexing fixed-size-1024-200
Indexing markdown-header
Indexing recursive-2048-0
Indexing fixed-size-2048-200
Indexing recursive-512-200
Indexing recursive-2048-200
Indexing recursive-1024-0
Indexing markdown-header-parent
Indexing fixed-size-512-200
Indexing semantic-chunks-95-recursive-2048-200
Indexing semantic-chunks-90
Indexing semantic-chunks-95
Indexing recursive-512-0
Indexing markdown-header-recursive-512-200


# 5. Evaluation


## Create Golden Datasets

3 Evaluation Golden Datasets for each Chunking Strategy should include the following:

- Questions across Documents
- Ground Truth Chunks (with graded Relevance)
- Ground Truth Answers

For Simple, Reasoning and Multi-Context Questions


In [7]:
from utils.evaluation import GoldenTestset

class Questions(TypedDict):
    simple: List[GoldenTestset]
    reasoning: List[GoldenTestset]
    multi_context: List[GoldenTestset]

gold_dataset: Dict[str, Questions]  = {}


Create golden dataset on subset of documents, to have some irrelevant documents left for some noise

In [8]:
documents_subset_sources = [data_dir+"documents/sleep.md", data_dir+"documents/teeth.md", data_dir+"documents/time_management.md", data_dir+"documents/mentoring.md"]

### Question Generation with RAGAS


Generate synthetic Questions across Documents to challenge chunking strategies on multi-context queries


In [9]:
from os import environ

environ["RAGAS_DO_NOT_TRACK"] = "true"

In [17]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

nest_asyncio.apply()

generator_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

generator = TestsetGenerator.from_langchain(generator_llm, critic_llm, embeddings)

ragas_testset = generator.generate_with_langchain_docs(
    [document for document in documents if document.metadata["source"] in documents_subset_sources],
    test_size=10,
    distributions={simple: 0.4, reasoning: 0.4, multi_context: 0.2},
)
df = ragas_testset.to_pandas()
df = df.drop(columns=["contexts"]) # ground truth contexts/chunks are determined in next step
df.to_csv(data_dir+"ragas_testset.csv", index=False)

embedding nodes:   0%|          | 0/34 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
ragas_testset = pd.read_csv(data_dir+"ragas_testset.csv")
for experiment_name in split_chunks.keys():
    if "markdown-header" not in experiment_name:
        continue
    gold_dataset[experiment_name] = {
        "simple": [],
        "reasoning": [],
        "multi_context": []
    }
    for _, row in ragas_testset.iterrows():
        testset = {
            "question": row['question'],
            "source": row['metadata'],
            "ground_truth_chunks": {},
            "ground_truth_answer": row['ground_truth']
        }
        gold_dataset[experiment_name][row["evolution_type"]].append(testset)

Skipping fixed-size-2048-0
Skipping recursive-1024-200
Skipping fixed-size-512-0
Skipping fixed-size-1024-0
Skipping fixed-size-1024-200
Skipping recursive-2048-0
Skipping fixed-size-2048-200
Skipping recursive-512-200
Skipping recursive-2048-200
Skipping recursive-1024-0
Skipping fixed-size-512-200
Skipping semantic-chunks-95-recursive-2048-200
Skipping semantic-chunks-90
Skipping semantic-chunks-95
Skipping recursive-512-0


### Generate Relevancy Score for each chunk


Relevancy Prompt is taken by Trulens. The difference is that I apply it to all chunks whereas Trulens only computed it on the retrieved chunks


In [18]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from utils.llm_output_parser import re_0_10_rating

system_prompt = """You are a RELEVANCE grader; providing the relevance of the given CONTEXT to the given QUESTION.
    Respond only as a number from 0 to 10 where 0 is the least relevant and 10 is the most relevant. 

    A few additional scoring guidelines:

    - Long CONTEXTS should score equally well as short CONTEXTS.

    - RELEVANCE score should increase as the CONTEXTS provides more RELEVANT context to the QUESTION.

    - RELEVANCE score should increase as the CONTEXTS provides RELEVANT context to more parts of the QUESTION.

    - CONTEXT that is RELEVANT to some of the QUESTION should score of 2, 3 or 4. Higher score indicates more RELEVANCE.

    - CONTEXT that is RELEVANT to most of the QUESTION should get a score of 5, 6, 7 or 8. Higher score indicates more RELEVANCE.

    - CONTEXT that is RELEVANT to the entire QUESTION should get a score of 9 or 10. Higher score indicates more RELEVANCE.

    - CONTEXT must be relevant and helpful for answering the entire QUESTION to get a score of 10.

    - Never elaborate."""

user_prompt = PromptTemplate.from_template(
    """QUESTION: {question}

    CONTEXT: {context}
    
    RELEVANCE: """
)

critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


def make_request_with_backoff(messages, retries=8):
    for i in range(retries):
        try:
            response = critic_llm.invoke(messages)
            return response
        except openai.RateLimitError as e:
            if i == retries - 1:
                raise e
            wait_time = 2**i
            print(f"Rate limited, waiting {wait_time} seconds")
            time.sleep(wait_time)
        except openai.APIError as e:
            print(e)


def process_chunk(chunk, testset):
    if chunk.metadata["source"] not in testset["source"]:
        return None, None

    judge_chunk_relevancy_prompt = user_prompt.format(
        question=testset["question"], context=chunk.page_content
    )

    llm_messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=judge_chunk_relevancy_prompt),
    ]
    response = make_request_with_backoff(llm_messages)
    chunk_relevancy = re_0_10_rating(response.content)
    if chunk_relevancy != 0.0:
        return str(chunk.metadata["id"]), chunk_relevancy
    return None, None

for experiment_name, questions in gold_dataset.items():
    if "markdown-header-recursive-512-200" not in experiment_name and "markdown-header-parent" not in experiment_name:
        continue
    print("Collecting ground truth for", experiment_name)
    for question_type, testsets in questions.items():
        print("Collecting ground truth for", question_type)
        for testset in tqdm(testsets):
            ground_truth = {}
            with ThreadPoolExecutor(max_workers=2) as executor:
                future_to_chunk = {
                    executor.submit(process_chunk, chunk, testset): chunk
                    for chunk in split_chunks[experiment_name]
                }
                for future in as_completed(future_to_chunk):
                    chunk_id, relevancy = future.result()
                    if chunk_id and relevancy:
                        ground_truth[chunk_id] = relevancy
            
            if len(ground_truth):
                testset["ground_truth_chunks"] = ground_truth

100%|██████████| 4/4 [00:48<00:00, 12.24s/it]


100%|██████████| 4/4 [01:02<00:00, 15.68s/it]


100%|██████████| 2/2 [00:23<00:00, 11.51s/it]


100%|██████████| 4/4 [00:14<00:00,  3.52s/it]


100%|██████████| 4/4 [00:17<00:00,  4.39s/it]


100%|██████████| 2/2 [00:06<00:00,  3.40s/it]


Save Evaluation Dataset


In [26]:
with open(data_dir+'gold_dataset_1.json', 'w') as jsonl_file:
    json.dump(gold_dataset, jsonl_file, indent=4)

## Evaluate Retrieval

Load Evaluation Dataset


In [8]:
gold_dataset = {}
with open(data_dir+'gold_dataset.json', 'r') as jsonl_file:
    gold_dataset = json.load(jsonl_file)

In [33]:
from utils.evaluation import calculate_metrics, calculate_mean_metrics

K = None
results = pd.DataFrame(columns=[f"Experiment@{K or 'Dyn'}", "Question Type", "Precision", "Recall", "MAP", "NDCG"])

for experiment_name, questions in gold_dataset.items():
    if experiment_name not in vector_stores:
        continue
    
    for question_type, testsets in questions.items():
        metrics = []
        for testset in testsets:
            question = testset["question"]
            ground_truth = testset["ground_truth_chunks"]
            retriever = vector_stores[experiment_name].as_retriever(search_kwargs={"k": K or len(ground_truth)})
            retrieved_chunks = retriever.invoke(question)
            retrieved_chunk_ids = [str(doc.metadata["id"]) for doc in retrieved_chunks]
            metrics.append(calculate_metrics(retrieved_chunk_ids, ground_truth_chunks=list(ground_truth.keys()), ground_truth_relevancies=list(ground_truth.values()), K=K or len(ground_truth)))
        
        mean_metrics = calculate_mean_metrics(metrics)
        
        results.loc[len(results)] = [
            experiment_name,
            question_type,
            mean_metrics["precision"],
            mean_metrics["recall"],
            mean_metrics["map"],
            mean_metrics["ndcg"]
        ]

results.to_csv(f"{data_dir}results/{K or 'Dyn'}-k.csv", index=False)

In [34]:
K = None
results = pd.read_csv(f"{data_dir}results/{K or 'Dyn'}-k.csv")
results_view = results.drop(columns=["Question Type"]).groupby(f"Experiment@{K or 'Dyn'}").mean().sort_values(by="NDCG", ascending=False)
results_view

,Precision,Recall,MAP,NDCG
Experiment@Dyn,,,,
recursive-2048-0,0.937616,0.937616,0.927859,0.962165
fixed-size-2048-0,1.000000,1.000000,1.000000,0.960288
recursive-2048-200,0.914881,0.914881,0.911222,0.953246
fixed-size-2048-200,0.945139,0.945139,0.944813,0.945893
markdown-header-parent,0.896152,0.896152,0.882449,0.945647
fixed-size-1024-0,0.908189,0.908189,0.899410,0.943800
fixed-size-1024-200,0.907985,0.907985,0.898566,0.943372
markdown-header,0.902953,0.902953,0.888976,0.936472
markdown-header-recursive-1024-0,0.898361,0.898361,0.877940,0.934421


## Evaluate Generation

In [ ]:

nest_asyncio.apply()

answer_correctness_system_prompt = """You are a CORRECTNESS grader; providing the correctness of the given GENERATED ANSWER compared to the given GROUND TRUTH ANSWER.
Respond only as a number from 0 to 10 where 0 is the least correct and 10 is the most correct.

A few additional scoring guidelines:

- Long GENERATED ANSWERS should score equally well as short GENERATED ANSWERS.

- CORRECTNESS score should increase as the GENERATED ANSWER matches more accurately with the GROUND TRUTH ANSWER.

- CORRECTNESS score should increase as the GENERATED ANSWER covers more parts of the GROUND TRUTH ANSWER accurately.

- GENERATED ANSWERS that partially match the GROUND TRUTH ANSWER should score 2, 3, or 4. Higher scores indicate more correctness.

- GENERATED ANSWERS that mostly match the GROUND TRUTH ANSWER should get a score of 5, 6, 7, or 8. Higher scores indicate more correctness.

- GENERATED ANSWERS that fully match the GROUND TRUTH ANSWER should get a score of 9 or 10. Higher scores indicate more correctness.

- GENERATED ANSWERS must be fully accurate and comprehensive to the GROUND TRUTH ANSWER to get a score of 10.

- Never elaborate."""

answer_correctness_user_prompt = PromptTemplate.from_template(
    """GROUND TRUTH ANSWER: {ground_truth_answer}

GENERATED ANSWER: {generated_answer}

CORRECTNESS: """
)

prompt = hub.pull("rlm/rag-prompt")
generator_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
        
for experiment_name, questions in gold_dataset.items():
    print("Evaluating", experiment_name)
    vector_stores[experiment_name].embeddings.show_progress_bar = False
    retriever = vector_stores[experiment_name].as_retriever(search_kwargs={"k": 10})
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | generator_llm
        | StrOutputParser()
    )
    for question_type, testsets in questions.items():
        mean_answer_correctness = 0
        for testset in testsets:
            response = rag_chain.invoke(testset["question"])
            answer_correctness_prompt = answer_correctness_user_prompt.format(
                ground_truth_answer=testset["ground_truth_answer"], generated_answer=response
            )

            llm_messages = [
                SystemMessage(content=answer_correctness_system_prompt),
                HumanMessage(content=answer_correctness_prompt),
            ]
            response = make_request_with_backoff(llm_messages)

            answer_correctness = re_0_10_rating(response.content)
            mean_answer_correctness += answer_correctness
        mean_answer_correctness /= len(testsets)
        print(f"Experiment: {experiment_name} Question Type: {question_type} Mean Answer Correctness: {mean_answer_correctness}")